In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

# 1) Download and Explore Data

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

## Create Data Frame for the neighborhoods of New York City

In [4]:
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [5]:
neighborhoods.shape

(306, 4)

## Use geopy library to get coordinates of New York

In [6]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


## Map of New York City inlcuding all neighborhoods

In [7]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# 2) Foursquare API

In [8]:
#define credentials
CLIENT_ID = 'FO2Q4DLGLFZCP2EWOAXRAYVAEFBTAOM5UNWSI3KLL550UIZ2' # your Foursquare ID
CLIENT_SECRET = 'VKN42NOF5TZD5Q2CX0AOVJTIBEWERR4RS5QNMWDSG15GLQEX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 500

## Search for all venues per neighborhood

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:

ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [11]:
print(ny_venues.shape)
ny_venues.head()

(9874, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [12]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,30,30,30,30,30,30
Annadale,15,15,15,15,15,15
Arden Heights,5,5,5,5,5,5
Arlington,6,6,6,6,6,6
Arrochar,20,20,20,20,20,20
Arverne,18,18,18,18,18,18
Astoria,92,92,92,92,92,92
Astoria Heights,13,13,13,13,13,13
Auburndale,18,18,18,18,18,18


## Analyze each neighborhood

In [13]:
# one hot encoding
ny_onehot = []
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood']
# move neighborhood column to the first column
temp = ny_onehot['Neighborhood']
ny_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
ny_onehot.insert(0, 'Neighborhood', temp)
ny_onehot.head()


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Offi

## Create grouped dataframe based on the number of Italian restaurants at every neighborhood

In [14]:
df_italian = ny_onehot[["Neighborhood","Italian Restaurant"]]

In [15]:
df_italian.head()
print(df_italian.shape)

(9874, 2)


In [28]:
df_grouped = df_italian.groupby('Neighborhood').count().reset_index()

In [17]:
df_grouped.head()

,Neighborhood,Italian Restaurant
0,Allerton,30
1,Annadale,15
2,Arden Heights,5
3,Arlington,6
4,Arrochar,20


# 3) Cluster Neighborhoods

In [25]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, init = "k-means++").fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 0, 4, 4, 1, 0, 4, 3], dtype=int32)

In [26]:
ny_data = neighborhoods.drop_duplicates(subset=['Neighborhood'])

In [29]:
df_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [30]:
df_grouped.head()

,Cluster Labels,Neighborhood,Italian Restaurant
0,4,Allerton,30
1,4,Annadale,15
2,0,Arden Heights,5
3,0,Arlington,6
4,4,Arrochar,20


In [31]:
ny_merged = ny_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(df_grouped.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Italian Restaurant
0,Bronx,Wakefield,40.894705,-73.847201,0,9
1,Bronx,Co-op City,40.874294,-73.829939,4,19
2,Bronx,Eastchester,40.887556,-73.827806,4,22
3,Bronx,Fieldston,40.895437,-73.905643,0,3
4,Bronx,Riverdale,40.890834,-73.912585,0,9


In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Clusters in detail

In [38]:
cluster1 = ny_merged[ny_merged['Cluster Labels'] == 0]
cluster2 = ny_merged[ny_merged['Cluster Labels'] == 1]
cluster3 = ny_merged[ny_merged['Cluster Labels'] == 2]
cluster4 = ny_merged[ny_merged['Cluster Labels'] == 3]
cluster5 = ny_merged[ny_merged['Cluster Labels'] == 4]


In [41]:
cluster1.sort_values(by=['Italian Restaurant'])

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Italian Restaurant
207,Staten Island,Port Ivory,40.639683,-74.174645,0,1
203,Staten Island,Todt Hill,40.597069,-74.111329,0,1
255,Staten Island,Emerson Hill,40.606794,-74.097762,0,1
193,Queens,Brookville,40.660003,-73.751753,0,1
192,Queens,Somerville,40.597711,-73.796648,0,1
257,Staten Island,Howland Hook,40.638433,-74.186223,0,1
183,Queens,Jamaica Estates,40.716805,-73.787227,0,2
76,Brooklyn,Mill Island,40.606336,-73.908186,0,2
3,Bronx,Fieldston,40.895437,-73.905643,0,3
202,Staten Island,Grymes Hill,40.624185,-74.087248,0,3


In [42]:
cluster2.sort_values(by=['Italian Restaurant'])

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Italian Restaurant
131,Queens,Jackson Heights,40.751981,-73.882821,1,83
107,Manhattan,Upper East Side,40.775639,-73.960508,1,83
247,Manhattan,Carnegie Hill,40.782683,-73.953256,1,84
46,Brooklyn,Bay Ridge,40.625801,-74.030621,1,86
16,Bronx,Fordham,40.860997,-73.896427,1,87
101,Manhattan,Washington Heights,40.851903,-73.936900,1,88
249,Manhattan,Civic Center,40.715229,-74.005415,1,88
49,Brooklyn,Greenpoint,40.730201,-73.954241,1,91
129,Queens,Astoria,40.768509,-73.915654,1,92
250,Manhattan,Midtown South,40.748510,-73.988713,1,92


In [44]:
cluster3.sort_values(by=['Italian Restaurant'])

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Italian Restaurant
153,Queens,Little Neck,40.770826,-73.738898,2,53
301,Manhattan,Hudson Yards,40.756658,-74.000111,2,54
272,Queens,Hunters Point,40.743414,-73.953868,2,56
283,Brooklyn,Dumbo,40.703176,-73.988753,2,57
279,Brooklyn,Fulton Ferry,40.703281,-73.995508,2,58
70,Brooklyn,Park Slope,40.672321,-73.977050,2,58
102,Manhattan,Inwood,40.867684,-73.921210,2,58
103,Manhattan,Hamilton Heights,40.823604,-73.949688,2,61
127,Manhattan,Battery Park City,40.711932,-74.016869,2,61
138,Queens,Flushing,40.764454,-73.831773,2,63


In [45]:
cluster4.sort_values(by=['Italian Restaurant'])

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Italian Restaurant
8,Bronx,Norwood,40.877224,-73.879391,3,32
268,Bronx,Concourse Village,40.824780,-73.915847,3,32
170,Queens,Far Rockaway,40.603134,-73.754980,3,32
30,Bronx,Parkchester,40.837938,-73.856003,3,33
31,Bronx,Westchester Square,40.840619,-73.842194,3,33
142,Queens,Maspeth,40.725427,-73.896217,3,33
47,Brooklyn,Bensonhurst,40.611009,-73.995180,3,34
262,Brooklyn,Mill Basin,40.615974,-73.915154,3,34
61,Brooklyn,Williamsburg,40.707144,-73.958115,3,34
197,Staten Island,St. George,40.644982,-74.079353,3,34


In [46]:
cluster5.sort_values(by=['Italian Restaurant'])

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Italian Restaurant
295,Brooklyn,Highland Park,40.681999,-73.890346,4,15
162,Queens,Queens Village,40.718893,-73.738715,4,15
264,Queens,Utopia,40.733500,-73.796717,4,15
208,Staten Island,Castleton Corners,40.613336,-74.119181,4,15
22,Bronx,Port Morris,40.801664,-73.913221,4,15
215,Staten Island,Annadale,40.538114,-74.178549,4,15
72,Brooklyn,East New York,40.669926,-73.880699,4,16
60,Brooklyn,Brownsville,40.663950,-73.910235,4,16
302,Queens,Hammels,40.587338,-73.805530,4,16
278,Queens,Blissville,40.737251,-73.932442,4,16


In [50]:
clusters_5 = {'Cluster': ['Red','Orange','Light Blue','Light Green','Purple'],
        'Italian Restaurants': ['0-14','15-30','31-52','53-80','80-130']
        }

df_clusters = pd.DataFrame(clusters_5, columns = ['Cluster', 'Italian Restaurants'])


In [51]:
df_clusters

,Cluster,Italian Restaurants
0,Red,0-14
1,Orange,15-30
2,Light Blue,31-52
3,Light Green,53-80
4,Purple,80-130
